In [1]:
import sys
sys.path.append('../code')
from init_mooc_nb import *
init_notebook()

Populated the namespace with:
np, matplotlib, kwant, holoviews, init_notebook, SimpleNamespace, pprint_matrix, scientific_number, pretty_fmt_complex, plt, pf, display_html
from code/edx_components:
MoocVideo, PreprintReference, MoocDiscussion, MoocCheckboxesAssessment, MoocMultipleChoiceAssessment, MoocPeerAssessment, MoocSelfAssessment
from code/functions:
spectrum, hamiltonian_array, h_k, pauli
Using kwant 1.3.2 and holoviews 1.9.5+g39fe834be
Executed on 2018-04-09 at 06:26:24.334338.


## Preamble Code

In [1]:
import sys
sys.path.append('../code')
from init_mooc_nb import *
init_notebook()

%opts Layout [sublabel_format='' aspect_weight=1 figure_size=(100) vspace=0.4]
%output size = 150

randn = np.random.randn

alphas = np.linspace(0, 1, 1000)

dims = SimpleNamespace(E=holoviews.Dimension(r'$E$'),
                       alpha=holoviews.Dimension(r'$\alpha$'),
                       Q=holoviews.Dimension(r'$Q$'),
                       Q_BdG=holoviews.Dimension(r'$Q_{BdG}$'))

def make_random_real_ham(N):
    H = randn(N, N)
    H += H.T
    return H / 2


def make_cons_ham(N):
    H = np.kron(pauli.s0, randn(N, N)) + np.kron(pauli.sz, randn(N, N))
    H += H.T.conj()
    return H / 2


def make_random_ham(N):
    H = randn(N, N) + 1j * randn(N, N)
    H += H.T.conj()
    return H / 2


def make_random_symplectic_ham(N):
    if N % 2:
        raise ValueError('Matrix dimension should be a multiple of 2')
    sy = np.kron(np.eye(N // 2), np.array([[0, -1j], [1j, 0]]))
    h = randn(N, N) + 1j * randn(N, N)
    h += h.T.conj()
    Th = sy @ h.conj() @ sy
    return (h + Th) / 4


def make_chiral_ham(N):
    temp1 = randn(N, N) + 1j * randn(N, N)
    temp2 = randn(N, N) + 1j * randn(N, N)    
    H = np.kron(pauli.sx, temp1) + np.kron(pauli.sy, temp2)
    H += H.T.conj()
    return H / 2


def make_BdG_ham(N):
    # This is antisymmetric basis
    H = 1j * randn(2*N, 2*N)
    H += H.T.conj()
    return H / 2


def energies(alpha, H0, H1):
    H = (1 - alpha) * H0 + alpha * H1
    return np.linalg.eigvalsh(H)


def find_spectrum(alphas, H0, H1):
    spectrum = [energies(a, H0, H1) for a in alphas]
    return np.array(spectrum)


def find_Q(spectrum):
    """Finds the number of bands that are under zero energy.

    Parameters:
    -----------
    spectrum : numpy array
        Array that contains the energies levels for every alpha.

    Returns:
    --------
    Q : list
        Number of bands under zero energy.
    """
    return [len(s[s<0]) for s in spectrum]


def plot_hamiltonian_spectrum(alphas, spectrum, E_range=(-4, 4)):
    """Function that plots a spectrum for a range of alphas.

    Parameters:
    -----------
    alphas : numpy array
        Range of alphas for which the energies are calculated.
    spectrum : numpy array
        Array that contains the energies levels for every alpha.
    E_range : tuple
        The upper and lower limit of the y-dimension.

    Returns:
    --------
    plot : holoviews.Path object
        Plot of alphas vs. spectrum.
    """
    E_min, E_max = E_range
    plot = holoviews.Path((alphas, spectrum), kdims=[dims.alpha, dims.E])[:, E_min:E_max] * holoviews.HLine(0)
    return plot(plot={'xticks':[0, 0.5, 1], 'yticks':[E_min, 0, E_max]})


def plot_Q(alphas, Q, Q_range, Q_dim=dims.Q):
    """Function that plots value of Q for a range of alphas.

    Parameters:
    -----------
    alphas : numpy array
        Range of alphas for which the energies are calculated.
    Q : numpy array
        Vector that contains the value of Q for every alpha.
    Q_range : tuple
        The upper and lower limit of the y-dimension.

    Returns:
    --------
    plot : holoviews.Path object
        Plot of alphas vs. Q.
    """
    Q_min, Q_max = Q_range
    Q_mid = (Q_max + Q_min) / 2
    plot = holoviews.Area((alphas, Q), kdims=[dims.alpha], vdims=[Q_dim])[:, Q_min:Q_max](style={'alpha': 0.4})
    plot *= holoviews.Curve((alphas, Q), kdims=[dims.alpha], vdims=[Q_dim])[:, Q_min:Q_max]
    return plot(plot={'xticks':[0, 0.5, 1], 'yticks':[Q_min, Q_mid, Q_max], 'aspect':4})

Populated the namespace with:
np, matplotlib, kwant, holoviews, init_notebook, SimpleNamespace, pprint_matrix, scientific_number, pretty_fmt_complex, plt, pf, display_html
from code/edx_components:
MoocVideo, PreprintReference, MoocDiscussion, MoocCheckboxesAssessment, MoocMultipleChoiceAssessment, MoocPeerAssessment, MoocSelfAssessment
from code/functions:
spectrum, hamiltonian_array, h_k, pauli
Using kwant 1.3.2 and holoviews 1.9.5+g39fe834be
Executed on 2018-04-09 at 06:26:35.613183.


In [14]:
%%opts Overlay [yticks=[-1, 0, 1]]

def find_pfaffian(alphas, H0, H1):
    """Function caculates the Pfaffian for a Hamiltonian.

    Parameters:
    -----------
    alphas : numpy array
        Range of alphas for which the energies are calculated.
    H0 : numpy array
        Hamiltonian, same size as H1.
    H1 : numpy array
        Hamiltonian, same size as H0.
        
    Returns:
    --------
    pfaffians : numpy array
        Pfaffians for each alpha.
    """
    def H(alpha):
        return (1 - alpha) * H0 + alpha * H1
    pfaffians = [np.sign(np.real(pf.pfaffian(1j*H(a)))) for a in alphas]
    return np.array(pfaffians)

pfaffian = find_pfaffian(alphas, H0, H1)
(plot_hamiltonian_spectrum(alphas, spectrum, [-1.5, 1.5]) + plot_Q(alphas, pfaffian, [-1.5, 1.5], dims.Q_BdG)).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Path.I  :Path   [$\alpha$,$E$]
      .HLine.I :HLine   [x,y]
   .Overlay.II :Overlay
      .Area.I  :Area   [$\alpha$]   ($Q_{BdG}$)
      .Curve.I :Curve   [$\alpha$]   ($Q_{BdG}$)

# Simulations: what about other symmetries

So you've made it through the content of the first week. Congratulations!

Now let's get our hands dirty. 

Let's begin by grabbing the notebooks of this week and the extra code we use to run these notebooks over [here](http://tiny.cc/topocm_smc). (Click the [i] button to the left of the folder that you want to copy.)

You need to copy the `code` folder and the `w1_topointro` folder. Let's look into what's inside.


## First task: combination of particle-hole and time-reversal symmetries

Look at the notebook about topology of zero-dimensional systems, and see how we generate Hamiltonians with a spinful time-reversal symmetry

$$
H = \sigma_y H^* \sigma_y.
$$

Now try to add this time reversal symmetry to a Hamiltonian which also has particle-hole symmetry. It is easiest to do in the basis where particle-hole symmetry has the form $H = -H^*$.
What do you think will happen? What will the extra symmetry do to the topological invariant?
Test your guess by plotting the spectrum and calculating the Pfaffian invariant.




### What do you think will happen?
The addition of spinful time-reversal symmetry will apply Kramer's degeneracy to the Hamiltonian's (ie. all energy levels will be doubly degenerate. Whereas previously it was possible to have zero energy crossings of particle-hole anti-symmetric pairs the symmetry enforces that each energy level be doubly degenerate (any zero-crossing causes a change of $\pm2$ and the Energy spectrum must be symmetric with regards to reflection around $E=0$

In [26]:
 
N = 8
H0 = make_random_symplectic_ham(N=N)
H1 = make_random_symplectic_ham(N=N)
spectrum = find_spectrum(alphas, H0, H1)
plot_hamiltonian_spectrum(alphas, spectrum)

:Overlay
   .Path.I  :Path   [$\alpha$,$E$]
   .HLine.I :HLine   [x,y]

### What will the extra symmetry do to the topological invariant?
The addition of the Time-reversal symmettry will cause the pfaffian to become invariant with respect to a smooth transition between Hamiltonians. This is a result of Kramer's degeneracy swapping the sign of 

### Test your guess by plotting the spectrum and calculating the Pfaffian invariant.

In [10]:
def make_random_symplectic_ham(N):
    if N % 2:
        raise ValueError('Matrix dimension should be a multiple of 2')
    sy = np.kron(np.eye(N // 2), np.array([[0, -1j], [1j, 0]]))
    h = randn(N, N) + 1j * randn(N, N)
    h += h.T.conj()
    Th = sy @ h.conj() @ sy
    return (h + Th) / 4



def make_BdG_ham(N):
    # This is antisymmetric basis
    H = 1j * randn(2*N, 2*N)
    H += H.T.conj()
    return H / 2


def make_time_reversed_particle_hole(N=None,bdg=None):
    if bdg is not None:
        N = bdg.shape[0]
    else:
        bdg = make_BdG_ham(N//2)
    if N % 2:
        raise ValueError('Matrix dimension should be a multiple of 2')
    sy = np.kron(np.eye(N // 2), np.array([[0, -1j], [1j, 0]]))
    h = bdg
    h += h.T.conj()
    Th = sy @ h.conj() @ sy
    return (h + Th) / 4
    
    
    

In [11]:
N = 4
bdg = make_BdG_ham(N)
pprint_matrix(bdg)

In [12]:
time_reversed_bdg = make_time_reversed_particle_hole(bdg=bdg)
pprint_matrix(time_reversed_bdg)

In [53]:
N = 32
H0 = make_time_reversed_particle_hole(N=N)
H1 = make_time_reversed_particle_hole(N=N)
spectrum = find_spectrum(alphas, H0, H1)
pfaffian = find_pfaffian(alphas, H0, H1)
(plot_hamiltonian_spectrum(alphas, spectrum, [-1.5, 1.5]) + plot_Q(alphas, pfaffian, [-1.5, 1.5], dims.Q_BdG)).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Path.I  :Path   [$\alpha$,$E$]
      .HLine.I :HLine   [x,y]
   .Overlay.II :Overlay
      .Area.I  :Area   [$\alpha$]   ($Q_{BdG}$)
      .Curve.I :Curve   [$\alpha$]   ($Q_{BdG}$)

## Second task: Su-Schrieffer-Heeger (SSH) model

Similar to the Kitaev chain, the SSH model is simple a one-dimensional model where you can see all the essential aspects of topological systems. Unlike the Kitaev chain it does correspond to a physical system: electrons in a polyacetylene chain.

Here's such a chain:

![](figures/polyacetylene.png)

Due to the dimerization of the chain the unit cell has two atoms and the hoppings have alternating strengths $t_1$ and $t_2$, so that the Hamiltonian is
$$H = \sum_{n=1}^N t_1 \left|2n-1\right\rangle\left\langle 2n\right|+t_2 \left|2n\right\rangle \left\langle 2n+1\right| + \textrm{h.c}$$

We can choose to start a unit cell from an even-numbered site, so $t_1$ becomes intra-cell hopping and $t_2$ inter-cell hopping.


Now get the notebook with the Kitaev chain and transform a Kitaev chain into an SSH chain.

Now repeat the calculations we've done with Majoranas using SSH chain. Keep $t_1 = 1$ and vary $t_2$.
You should see something very similar to what you saw with the Kitaev chain.

As you can guess, this is because the chain is topological.
Think for a moment: what kind of symmetry protects the states at the edges of the chain.
*(Hint: you did encounter this symmetry in our course.)*

The particle-hole symmetry, is a consequence of a mathematical transformation, and cannot be broken.
The symmetry protecting the SSH chain, however, can be broken.
Test your guess about the protecting symmetry by adding to your chain a term which breaks this symmetry and checking what it does to the spectrum of a finite chain and to its dispersion (especially as chain goes through a phase transition).

In [ ]:
MoocSelfAssessment()

**Now share your results:**

In [ ]:
MoocDiscussion('Labs', 'Toy models simulations')

# Review assignment

For the first week we have these papers:

In [ ]:
display_html(PreprintReference('1103.0780', 
  description="Topological classification is not always applied to Hamiltonians. "
              "Figure out what is the topological quantity in open systems. "
              "See this idea also applied in <a href=http://arxiv.org/abs/1405.6896>"
              "http://arxiv.org/abs/1405.6896</a>"))
display_html(PreprintReference('1305.2924', description="This is a study of statistical properties "
                                       "of topological transitions."))
display_html(PreprintReference('1111.6600', description="A toy model may still be useful in practice."))

### Bonus: Find your own paper to review!

Do you know of another paper that fits into the topics of this week, and you think is good?
Then you can get bonus points by reviewing that paper instead!

In [ ]:
MoocPeerAssessment()

**Do you have questions about what you read? Would you like to suggest other papers? Tell us:**

In [ ]:
MoocDiscussion("Reviews", "Toy models")